## SEARCH FOR DM SIGNALS FROM ELLIPTICAL GALAXIES - A STACKING ANALYSIS
A. Steinhebel, GSFC, 2022

#### FIRST STEP OF ANALYSIS PIPELINE: Consider catalog(s) of elliptical galaxies and remove those not suitable for analysis. Require that:
|b|>15deg ; >1deg separation between EG and blazar (sources from BZCAT catalog) ; >1 deg separation between EG and radio galaxy (sources from 2MRS catalog)

##### Import necessary packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import astropy
from astropy import units as u
from astropy.coordinates import SkyCoord
import math
from astropy.io import fits
from astropy.table import Table
from os.path import exists

##### Define helpful functions 

In [2]:
def galacticPlaneOverlap(egDF, deg:float=15.):
#Remove candidates too close to galactic plane

    print("Removing EGs with b<15deg")
    removed=0
    toRemove=[]
    for x in egDF.index:
        if abs(egDF.loc[x,'b'])<deg:
            toRemove.append(x) #remove row if b<15deg
            removed+=1
    print(f"    Removed {removed} EGs - left with {(len(egDF.index)-removed)/len(egDF.index)*100:.2f}% of sample")
    return toRemove

In [3]:
def compCat(compDFName, raIn, decIn, egDF, sep:float=1.0):

    compDF=pd.read_csv(compDFName)
    toRemove=[]
    cl1=SkyCoord(l = egDF['l']*u.deg, b = egDF['b']*u.deg, frame='galactic').icrs
    cl2 = SkyCoord(ra = compDF[raIn]*u.degree,dec = compDF[decIn]*u.degree)

    #return arrays of elements from cl2 that match cl1
    #index of cl2 object closest to the corresponding cl1 object, 2D distance between objects, 3D distance between object (if exists)
    idx, d2d, d3d = cl1.match_to_catalog_sky(cl2)

    #Max separation between EG and other source
    max_sep = sep*u.degree #Maximum distance to neighbor
    sep_constraint = d2d < max_sep #array of booleans corresponding to cl1 - True if sep too small and cl1 element should be removed
    print(f"    Eliminating {sep_constraint.sum()} EGs for overlap within {sep} deg")

    for i in range(len(egDF)):
        if sep_constraint[i]:
            toRemove.append(i)

    return toRemove

In [4]:
def saveFromInput(saveFile, df, inputO:bool=False): 
    if inputO:
        inp= 'a'
        while inp!='y' and inp!='n':
            print("Overwrite? y/n")
            inp = input()
        savePlt=True if inp=='y' else False	
    else:
        savePlt=True

    if savePlt: 
        print(f"Saving the remaining EGs to {saveFile}")
        df.to_csv(saveFile, index=False)

### Import and format EG catalog #1 - KH (https://arxiv.org/pdf/1304.7762.pdf) 

In [5]:
egDF=pd.read_csv('EGs_KH.csv')

#get l,b from other file
egDF_mn=pd.read_csv('EGs_KH_lb.csv')
egDF_mn[['Name','Type','Distance[Mpc]','Distance Uncert','Ks','M_KsT','M_VT','(V-Ks)_0','(B-V)_0','M_BH[Msolar*10e6]','Flags:M','Flags:C','Flags:M_BH','l','b']]=egDF_mn['Name;Type;Distance[Mpc];Ks;M_KsT;M_VT;(V-Ks)_0;(B-V)_0;M_BH[Msolar*10e6];Flags:M;Flags:C;Flags:M_BH;;l;b'].str.split(';', expand=True)
egDF_mn.drop(columns=['Name;Type;Distance[Mpc];Ks;M_KsT;M_VT;(V-Ks)_0;(B-V)_0;M_BH[Msolar*10e6];Flags:M;Flags:C;Flags:M_BH;;l;b'], inplace=True)
egDF['l']=egDF_mn['l']
egDF['b']=egDF_mn['b']
#Convert l,b to floats
toConvert=['l','b']
for colu in toConvert:
    egDF[colu]=pd.to_numeric(egDF[colu])

In [6]:
#Remove rows of NaN entries
egDF.dropna(inplace=True)
totEGs=len(egDF.index)

In [7]:
#Add RA/Dec in degrees to egDF
c=SkyCoord(l = egDF['l']*u.deg, b = egDF['b']*u.deg, frame='galactic')
eg_ra=[]
eg_dec=[]
for coord in c.icrs: #convert galactic coordinates to icrs RA/Dec
    strcoord=coord.to_string()
    eg_ra.append(float(strcoord.split(' ')[0]))
    eg_dec.append(float(strcoord.split(' ')[1]))
egDF['RA']=eg_ra
egDF['Dec']=eg_dec

cl1 = c.icrs #egDF SkyCoord object

Save elements to remove in an array - at the end of the script, remove these elements (allows for simpler iteration)

In [8]:
toRemove=[]

##### Compare to galactic plane

In [9]:
toRemove.append(galacticPlaneOverlap(egDF))

Removing EGs with b<15deg
    Removed 2 EGs - left with 95.56% of sample


##### Compare to BZCAT Blazar catalog (https://www.ssdc.asi.it/bzcat5/) 


Define separation value to be considered overlap

In [10]:
sep=1.0 
max_sep = sep*u.degree #Maximum distance to neighbor

In [11]:
#Identify closest source and mark for removal if separation<sep
toRemove.append(compCat('bzcat_blazarCatalog.csv',' R.A. (J2000) ', ' Dec. (J2000) ', egDF))

    Eliminating 17 EGs for overlap within 1.0 deg


##### Compare to 2MRS Radio galaxy catalog (http://ragolu.science.ru.nl/index.html))

In [12]:
#Identify closest source and mark for removal if separation<sep
toRemove.append(compCat('2mrs_radioCatalog.csv','ra', 'dec', egDF))

    Eliminating 19 EGs for overlap within 1.0 deg


##### Remove all EGs marked for removal

In [13]:
toRemove=sum(toRemove, []) #flatten list
toRemove = list(set(toRemove)) #remove duplicates
toRemove.sort()
for i in toRemove:
    egDF.drop(i,inplace=True)

print(f"Left with {len(egDF)} of the original {totEGs} EGs - {100*len(egDF)/totEGs:.2f}%")

Left with 17 of the original 45 EGs - 37.78%


##### Save resulting EG list as new csv

In [14]:
finalList="EGs_KH_overlapRemoved.csv"
saveFromInput(finalList,egDF,exists(finalList))

Overwrite? y/n
n


##### Clear catalog-specific variables

In [15]:
del egDF
del totEGs
del toRemove
del finalList

### Import EG catalog #2 - DF (https://academic.oup.com/mnras/article/460/4/4492/2609151)

Catalog should have been prepared previously with tables_DF/makeCSV.py . The resulting EGs_DF.csv should be moved to this directory. No formatting needed - all done in the creation step with tables_DF/makeCSV.py

In [16]:
egDF=pd.read_csv('EGs_DF.csv')
totEGs=len(egDF)

Save elements to remove in an array - at the end of the script, remove these elements (allows for simpler iteration)

In [17]:
toRemove=[]

##### Compare to galactic plane

In [18]:
toRemove.append(galacticPlaneOverlap(egDF))

Removing EGs with b<15deg
    Removed 5 EGs - left with 99.71% of sample


##### Compare to BZCAT Blazar catalog (https://www.ssdc.asi.it/bzcat5/) 


In [19]:
#Identify closest source and mark for removal if separation<sep
toRemove.append(compCat('bzcat_blazarCatalog.csv',' R.A. (J2000) ', ' Dec. (J2000) ', egDF))

    Eliminating 728 EGs for overlap within 1.0 deg


##### Compare to 2MRS Radio galaxy catalog (http://ragolu.science.ru.nl/index.html))

In [20]:
#Identify closest source and mark for removal if separation<sep
toRemove.append(compCat('2mrs_radioCatalog.csv','ra', 'dec', egDF))

    Eliminating 954 EGs for overlap within 1.0 deg


##### Remove all EGs marked for removal

In [21]:
toRemove=sum(toRemove, []) #flatten list
toRemove = list(set(toRemove)) #remove duplicates
toRemove.sort()
for i in toRemove:
    egDF.drop(i,inplace=True)

print(f"Left with {len(egDF)} of the original {totEGs} EGs - {100*len(egDF)/totEGs:.2f}%")

Left with 466 of the original 1715 EGs - 27.17%


##### Save resulting EG list as new csv

In [22]:
finalList="EGs_DF_overlapRemoved.csv"
saveFromInput(finalList,egDF,exists(finalList))

Overwrite? y/n
n
